# Trim Video duration

In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

def trim_video(input_path, output_path, duration=300):
    """
    Trims the video to the specified duration and saves it to a new file.

    :param input_path: Path to the input video file.
    :param output_path: Path to save the trimmed video file.
    :param duration: Duration in seconds to keep from the start of the video. Default is 300 seconds (5 minutes).
    """
    # Load the video file
    video = VideoFileClip(input_path)
    
    # Trim the video to the first 'duration' seconds
    trimmed_video = video.subclip(0, duration)
    
    # Write the trimmed video to a new file
    trimmed_video.write_videofile(output_path, codec="libx264")


input_video_path = "v1.mp4"
output_video_path = "v1_5min.mp4"
trim_video(input_video_path, output_video_path, 300)

# Export Video frames

In [ ]:
import cv2
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def save_frame(frame, frame_filename):
    cv2.imwrite(frame_filename, frame)

def video_to_frames(video_path, output_folder=None, max_workers=6, frame_skip=0):
    start_time = time.time()
    
    # Get the base name of the video file (without extension)
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    
    # Set the default output folder if not provided
    if output_folder is None:
        output_folder = os.path.join(os.getcwd(), video_name)
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    
    # Check if the video was opened successfully
    if not video_capture.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    frame_count = 0
    saved_frame_count = 0
    futures = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        while True:
            # Read the next frame from the video
            success, frame = video_capture.read()
            
            # If reading a frame was not successful, break the loop
            if not success:
                break
            
            # Save the frame if it's not being skipped
            if frame_count % (frame_skip + 1) == 0:
                # Construct the output file path
                frame_filename = os.path.join(output_folder, f"frame_{saved_frame_count:04d}.png")
                
                # Submit the frame saving task to the thread pool
                futures.append(executor.submit(save_frame, frame, frame_filename))
                
                saved_frame_count += 1
            
            frame_count += 1
    
    # Wait for all futures to complete
    for future in as_completed(futures):
        future.result()
    
    # Release the video capture object
    video_capture.release()
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Extracted {saved_frame_count} frames to {output_folder} in {total_time:.2f} seconds")



video_to_frames('v1.mp4', frame_skip=20)

In [ ]:
import winshell
import keyboard

# Function to restore the last deleted item from the Recycle Bin
def restore_last_deleted_item():
    try:
        # Get the list of items in the Recycle Bin
        items = list(winshell.recycle_bin())
        
        if items:
            # Restore the last deleted item
            last_item = items[-1]
            last_item.restore()
            print(f"Successfully restored: {last_item.name}")
        else:
            print("No items found in the Recycle Bin.")
    except Exception as e:
        print(f"Failed to restore the last deleted item: {e}")

# Set up the hotkey to restore the last deleted item
hotkey = 'numlock'
keyboard.add_hotkey(hotkey, restore_last_deleted_item)

print(f"Press {hotkey} to restore the last deleted item from the Recycle Bin.")

# Keep the script running
keyboard.wait('esc')

# CSV to YOLO text annotations

In [ ]:
import os
import pandas as pd

# Read the CSV data
data = pd.read_csv('labels_valomalo_2024-08-09-12-55-25.csv')

# Map class names to class IDs
class_mapping = {
    'Body': 0,
    'Head': 1
}

# Function to normalize coordinates
def normalize_bbox(x, y, width, height, img_width, img_height):
    x_center = (x + width / 2) / img_width
    y_center = (y + height / 2) / img_height
    norm_width = width / img_width
    norm_height = height / img_height
    return x_center, y_center, norm_width, norm_height

# Directory to save the annotation files
output_dir = 'annotations'
os.makedirs(output_dir, exist_ok=True)

# Process each row and write to corresponding text file
for _, row in data.iterrows():
    class_id = class_mapping[row['label_name']]
    x_center, y_center, norm_width, norm_height = normalize_bbox(
        row['bbox_x'], row['bbox_y'], row['bbox_width'], row['bbox_height'],
        row['image_width'], row['image_height']
    )
    
    # Format the annotation to six decimal places
    annotation = f"{class_id} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}\n"
    
    # Write to the corresponding text file in the specified directory
    txt_file_name = os.path.join(output_dir, row['image_name'].replace('.jpg', '.txt'))
    with open(txt_file_name, 'a') as f:
        f.write(annotation)

## Viz images & labels

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import random

def get_label_color(label, label_colors):
    if label not in label_colors:
        label_colors[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return label_colors[label]

def visualize_images_with_labels(image_folder, label_folder):
    label_colors = {}
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        label_path = os.path.join(label_folder, os.path.splitext(image_file)[0] + '.txt')
        
        image = cv2.imread(image_path)
        height, width, _ = image.shape
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    label = int(parts[0])
                    x_center, y_center, w, h = map(float, parts[1:])
                    
                    x_center *= width
                    y_center *= height
                    w *= width
                    h *= height
                    
                    x1 = int(x_center - w / 2)
                    y1 = int(y_center - h / 2)
                    x2 = int(x_center + w / 2)
                    y2 = int(y_center + h / 2)
                    
                    color = get_label_color(label, label_colors)
                    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        
        # Print the image name to the console
        print(f"{image_file}")
        
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()


image_folder = 'cutout_labels_640by640'
label_folder = 'adjusted_labels_640by640'
num_classes = 2
visualize_images_with_labels(image_folder, label_folder)

# Extract a Cutout of the Image containing the Labels + save the new adjust labels

In [31]:
import os
import random
from PIL import Image

def extract_cutout(image_path, label_path, label_number, cutout_size):
    image = Image.open(image_path)
    width, height = image.size
    cutout_width, cutout_height = cutout_size

    # Ensure cutout size is not larger than the image dimensions
    cutout_width = min(cutout_width, width)
    cutout_height = min(cutout_height, height)

    labels = []

    # If label file exists, process it
    if os.path.exists(label_path):
        with open(label_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                parts = line.strip().split()
                label = int(parts[0])
                x_center = float(parts[1]) * width
                y_center = float(parts[2]) * height
                box_width = float(parts[3]) * width
                box_height = float(parts[4]) * height

                if label == label_number:
                    # Calculate the cutout coordinates
                    left = int(x_center - cutout_width / 2)
                    top = int(y_center - cutout_height / 2)
                    right = left + cutout_width
                    bottom = top + cutout_height

                    # Ensure the cutout is within image bounds
                    left = max(0, min(left, width - cutout_width))
                    top = max(0, min(top, height - cutout_height))

                    cutout = image.crop((left, top, right, bottom))

                    # Adjust all label coordinates within the cutout region
                    for line in lines:
                        parts = line.strip().split()
                        label = int(parts[0])
                        x_center = float(parts[1]) * width
                        y_center = float(parts[2]) * height
                        box_width = float(parts[3]) * width
                        box_height = float(parts[4]) * height

                        if left <= x_center <= right and top <= y_center <= bottom:
                            new_x_center = (x_center - left) / cutout_width
                            new_y_center = (y_center - top) / cutout_height
                            new_box_width = box_width / cutout_width
                            new_box_height = box_height / cutout_height

                            labels.append((label, new_x_center, new_y_center, new_box_width, new_box_height))
                    return cutout, labels
    else:
        # If no labels, extract a random region from the middle
        if width > cutout_width and height > cutout_height:
            left_min = max(0, width // 4)
            left_max = min(width - cutout_width, 3 * width // 4)
            top_min = max(0, height // 4)
            top_max = min(height - cutout_height, 3 * height // 4)

            left = random.randint(left_min, left_max)
            top = random.randint(top_min, top_max)
        else:
            left = (width - cutout_width) // 2
            top = (height - cutout_height) // 2

        right = left + cutout_width
        bottom = top + cutout_height

        cutout = image.crop((left, top, right, bottom))
        return cutout, labels

def process_images(image_folder, label_folder, label_number, cutout_size, output_folder, label_output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    if not os.path.exists(label_output_folder):
        os.makedirs(label_output_folder)

    for image_name in os.listdir(image_folder):
        if image_name.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, image_name)
            label_path = os.path.join(label_folder, os.path.splitext(image_name)[0] + '.txt')
            cutout, labels = extract_cutout(image_path, label_path, label_number, cutout_size)
            if cutout:
                cutout.save(os.path.join(output_folder, image_name))
                label_output_path = os.path.join(label_output_folder, os.path.splitext(image_name)[0] + '.txt')
                with open(label_output_path, 'w') as label_file:
                    if labels:
                        for label in labels:
                            label_file.write(f"{label[0]} {label[1]:.6f} {label[2]:.6f} {label[3]:.6f} {label[4]:.6f}\n")
                    else:
                        # Write an empty file for images with no labels
                        label_file.write("")

image_folder = 'selected_images'
label_folder = 'labels_my-project-name_2024-08-09-08-42-56'
label_number = 0
cutout_size = (640, 640)  
output_folder = 'cutout_labels_640by640'
label_output_folder = 'adjusted_labels_640by640'

process_images(image_folder, label_folder, label_number, cutout_size, output_folder, label_output_folder)

# Split the main folder into TvT

In [32]:
import os
import random
import shutil

dataset_path = 'dataset'
images_path = os.path.join(dataset_path, 'images')
labels_path = os.path.join(dataset_path, 'labels')

# Preprocessing step: Delete .txt files that do not correspond to any image
label_files = [f for f in os.listdir(labels_path) if f.endswith('.txt')]
for label_file in label_files:
    image_file = label_file.replace('.txt', '.jpg')
    if not os.path.exists(os.path.join(images_path, image_file)):
        os.remove(os.path.join(labels_path, label_file))

# Get all image files
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]

# Shuffle the files
random.shuffle(image_files)

# Calculate split indices
total_images = len(image_files)
train_split = int(total_images * 0.7)
val_split = int(total_images * 0.2)

# Split the files
train_files = image_files[:train_split]
val_files = image_files[train_split:train_split + val_split]
test_files = image_files[train_split + val_split:]

# Copy files to their directories
def copy_files(file_list, dest_images_path, dest_labels_path):
    os.makedirs(dest_images_path, exist_ok=True)
    os.makedirs(dest_labels_path, exist_ok=True)
    for file in file_list:
        shutil.copy(os.path.join(images_path, file), dest_images_path)
        label_file = file.replace('.jpg', '.txt')
        if os.path.exists(os.path.join(labels_path, label_file)):
            shutil.copy(os.path.join(labels_path, label_file), dest_labels_path)

# Create directories and copy files
copy_files(train_files, 'yolo_dataset/train/images', 'yolo_dataset/train/labels')
copy_files(val_files, 'yolo_dataset/val/images', 'yolo_dataset/val/labels')
copy_files(test_files, 'yolo_dataset/test/images', 'yolo_dataset/test/labels')